# À propos de ce livret


### But du livret
Préparer et sauvegarder les fichiers de données pour une analyse subséquente. Données à partir du dossier «data».

### Différents scénarios
- Différents formats de données
    - Données qui ont très peu de lignes ou de colonnes
    - Données avec manquants ou sans manquants au préalable
- Différents patrons de manquants
    - Manquants de manière aléatoire
    - Manquants propres au domaine



### Notes sur le livret




## Préparation préalable

In [1]:
# Utilités
import os
import numpy as np
import pandas as pd

# Chargement des données
import scipy.io

# Barre de progression
from tqdm.auto import trange
from tqdm import tqdm_notebook, notebook

# Traitement préalable

Mettre sous le même format tous les jeux de données


Pour l'instant, le format *.mat* 

In [2]:
dossier_donnees = "data"

In [3]:
dossier_pems = "/transport_PeMS"
pems_entree = "/pems.npy"
pems_sortie = "/pems_raw.mat"

PeMS = np.load(dossier_donnees + dossier_pems + pems_entree)
scipy.io.savemat(dossier_donnees + dossier_pems + pems_sortie, {"mat": PeMS}) # Données non modifiées, donc brutes

In [4]:
dossier_seattle = "/transport_Seattle"
seattle_entree = "/mat.csv"
seattle_sortie = "/seattle_raw.mat"


seattle = pd.read_csv(dossier_donnees + dossier_seattle + seattle_entree, index_col=0)
seattle = seattle.to_numpy()
scipy.io.savemat(dossier_donnees + dossier_seattle + seattle_sortie, {"mat": seattle})

In [5]:
dossier_electricite = "/electricite"
electricite_entree = "/database_electricite_tronquee.csv"
electricite_sortie = "electricite_raw.mat"


elec = pd.read_csv(dossier_donnees + dossier_electricite + electricite_entree, sep=";")
elec = elec.to_numpy()
scipy.io.savemat(dossier_donnees + dossier_electricite + electricite_sortie, {"mat": elec})

## Formater les données

In [6]:
def format_donnees(dossier,
                   fichier_entree,
                   key_name,
                   format_size,
                  fichier_sortie,
                   retourner=False):
    """
    Mettre tous les fichiers sous forme de matrice et choisir les dimensions de la matrice
    
    
    Paramètres en entrée
    
        dossier : Nom du dossier où se trouve le jeu de données
        fichier : Nom du jeu de données
        key_name : nom de la clé du dictionnaire (spécifique au format .mat)
        format_size : format en  pourcentage du jeu de données (sous forme 1.0, 0.9, etc.)
        nom_nouv_fichier : nom du nouveau fichier créé (sans l'extension)
        retourner : choix de retourner ou non un objet à la fin de la fonction (à voir)
        
        
        Note : Les noms de clés des nouveaux fichiers sont "mat" par souci d'uniformation

    """
    # S'assurer des bons paramètres
    format_size = np.round(format_size, decimals = 1)
    
    if format_size > 1.0:
        format_size = 1.0
        
    
    # Charger les données
    donnees = scipy.io.loadmat(dossier + fichier)
    donnees = donnees[key_name]

    # Indépendemment du nombre de dimensions, mettre sous forme de matrice
    dimensions = donnees.shape
    dim1 = dimensions[0]
    donnees = donnees.reshape(dim1, -1)

    # Tronquer ou non les données (format_size)
    dim1, dim2 = donnees.shape
    dim1 = int(np.round(dim1 * format_size))
    dim2 = int(np.round(dim2 * format_size))

    donnees = donnees[:dim1, :dim2]

    # Indiquer le format dans le nom (?) - voir si pertinent
    nom_format = str(int(format_size * 100))

    # Sauvegarder le fichier avec les nouvelles dimensions
    nom_fichier = "{:}{:}_{:}.mat".format(dossier ,fichier_sortie, nom_format)
    scipy.io.savemat(nom_fichier,
                     {"mat": donnees})

    return donnees if retourner == True else None  # Choix de retourner ou non un objet dans la session

In [7]:
dossier = "data/transport_Seattle/"
fichier = "seattle_raw.mat"
fichier_sortie = "seattle" # Nom du fichier en sortie (nom de la ville)
key_name = "mat"
format_size = 0.5 
#nom_nouv_fichier = "seattle_mat" 
retourner = False ## À voir si pertinent

format_donnees(dossier, fichier, key_name, format_size, fichier_sortie, retourner)

## Création de la matrice binaire

In [8]:
def fichier_binaire(dossier,
                    fichier,
                    key_name,
                    prop_manquants,
                    seed=2020,
                    retourner=False):
    """
    Créer une matrice binaire à partir d'un fichier de données existant
    
    
    Paramètres en entrée
    
        dossier : Nom du dossier où se trouve le jeu de données
        fichier : Nom du jeu de données
        key_name : nom de la clé du dictionnaire (spécifique au format .mat)
        prop_manquants : format en  pourcentage du jeu de données (sous forme 1.0, 0.9, etc.)
        seed : noyau pour la reproductibilité des analysis
        retourner : choix de retourner ou non un objet à la fin de la fonction (à voir)
        
        
        

    """

    # S'assurer des bons paramètres
    prop_manquants = np.round(prop_manquants, decimals=2)

    if prop_manquants > 1.0:
        prop_manquants = 1.0

    # Charger les données (pour obtenir les dimensions de la matrice)
    donnees = scipy.io.loadmat(dossier + fichier)
    donnees = donnees[key_name]

    # Scénario de manquants
    p0 = prop_manquants  # taux de manquants
    p1 = 1 - p0  # taux de présents

    dimensions = donnees.shape

    np.random.seed(seed)
    binmat = np.random.choice([0, 1], size=dimensions, p=[p0, p1])
    
    
    ## Enregistrer la nouvelle matrice
    nom_nouv_fichier = "_{:}".format(str(int(p0 * 100)))
    nom_nouv_fichier = dossier + fichier + nom_nouv_fichier + ".mat"

    # Note : Les proportions peuvent varier légèrement, donc montrer la proportion de manquants à l'utilisateur
    dim1, dim2 = dimensions
    prop_finale = len(np.where(binmat == 0)[0]) / (dim1 * dim2)
    print(("Fichier : {0:} \nProportion de manquants : {1:.2%}".format(nom_nouv_fichier, prop_finale)))

    scipy.io.savemat(nom_nouv_fichier, {"mat": binmat})

    return binmat if retourner == True else None  # Choix de retourner ou non un objet dans la session

In [9]:
# Sous forme d'itérations
dossier = "data/transport_Seattle"
fichier = "/seattle_50"
key_name = "mat"
retourner = False  ## À voir si pertinent
seed = 2020

prop_manquants = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

for p in prop_manquants:
    fichier_binaire(dossier,
                    fichier,
                    key_name,
                    p, # proportion de manquants
                    seed=2020,
                    retourner=False)

Fichier : data/transport_Seattle/seattle_50_0.mat 
Proportion de manquants : 0.00%
Fichier : data/transport_Seattle/seattle_50_10.mat 
Proportion de manquants : 10.02%
Fichier : data/transport_Seattle/seattle_50_20.mat 
Proportion de manquants : 20.05%
Fichier : data/transport_Seattle/seattle_50_30.mat 
Proportion de manquants : 30.10%
Fichier : data/transport_Seattle/seattle_50_40.mat 
Proportion de manquants : 40.07%
Fichier : data/transport_Seattle/seattle_50_50.mat 
Proportion de manquants : 50.09%
Fichier : data/transport_Seattle/seattle_50_60.mat 
Proportion de manquants : 60.10%
Fichier : data/transport_Seattle/seattle_50_70.mat 
Proportion de manquants : 70.00%
Fichier : data/transport_Seattle/seattle_50_80.mat 
Proportion de manquants : 80.10%
Fichier : data/transport_Seattle/seattle_50_90.mat 
Proportion de manquants : 90.09%
